In [2]:
#% ignore
"""
This compiles all of a notebook's code cells into a single file.
"""

"\nThis compiles all of a notebook's code cells into a single file.\n"

In [3]:
#% compile
import json

CJN_FLAG_PREFIX = "#%"
CJN_FLAG_IGNORE = CJN_FLAG_PREFIX + " " + "IGNORE"
CJN_FLAG_COMPILE = CJN_FLAG_PREFIX + " " + "COMPILE"

CJN_OPT_COMPILE_MARKED = "compile_marked"
CJN_OPT_ENFORCE_EXEC_COUNT = "enforce_exec_count"

In [4]:
#% compile
def _read_json(fpath):
    with open(fpath, 'r') as f:
        s = f.read()
    return json.loads(s)

In [5]:
#% ignore
ex = _read_json('CJN.ipynb')
ex['cells'][1]['execution_count']

55

In [6]:
#% compile
def _clean_cell_head(cell):
    return cell['source'][0].upper().strip()

def _cell_can_compile(cell, opts):
    if not cell['cell_type']=='code':
        return False
    if len(cell['source'])==0:
        return False
    if _clean_cell_head(cell)==CJN_FLAG_IGNORE:
        return False
    if CJN_OPT_COMPILE_MARKED in opts:
        if opts[CJN_OPT_COMPILE_MARKED]:
            return _clean_cell_head(cell)==CJN_FLAG_COMPILE
    return True

def compile_notebook(notebook, output, opts={}):
    nb = _read_json(notebook)
    code = ''
    
    def build_exec_count_fn(opts):
        if CJN_OPT_ENFORCE_EXEC_COUNT in opts:
            if opts[CJN_OPT_ENFORCE_EXEC_COUNT]:
                return lambda c_p: (c_p[0]['execution_count'] > c_p[1], c_p[0]['execution_count'])
        return lambda c_p: True, None
    
    exec_count_check = build_exec_count_fn(opts)
    
    p = 0
    for cell in nb['cells']:
        if _cell_can_compile(cell, opts):
            contig, p = exec_count_check((cell, p))
            if not contig:
                raise Exception("Cell execution count was not contiguous.")
            
            for i in range(len(cell['source'])):
                line = cell['source'][i]
                if i > 0 or line[0:2] != CJN_FLAG_PREFIX:
                    code += line
            code += '\n\n'
    
    with open(output, 'w') as f:
        f.write(code)

In [7]:
#% compile
def load_compscript(compscript):
    return _read_json(compscript)

def save_compscript(script_def, compscript):
    j = json.dumps(script_def)
    with open(output, 'w') as f:
        f.write(j)

def append_compscript(file_defs, compscript):
    j = load_compscript(compscript)
    for fd in file_defs.keys():
        j[fd] = file_defs[fd]
    save_compscript(j, compscript)

In [8]:
#% ignore
ex_compilescript = {
    'CJN.ipynb': {
        'output': 'CJN.py',
        'opts': { 'compile_marked': True, 'enforce_exec_count': True }
    }
}

json.dumps(ex_compilescript)

'{"CJN.ipynb": {"output": "CJN.py", "opts": {"compile_marked": true, "enforce_exec_count": true}}}'

In [9]:
#% compile
def run_compile_script(compscript):
    cs = load_compscript(compscript)
    
    for f in cs.keys():
        compile_notebook(f, cs[f]['output'], cs[f]['opts'])

In [10]:
#% ignore
nb = _read_json("CJN.ipynb")
checker = lambda c_p: (c_p[0]['execution_count'] > c_p[1], c_p[0]['execution_count'])
p = 0
for cell in nb['cells']:
    if _cell_can_compile(cell, {'compile_marked': True}):
        contig, p = checker((cell, p))
        print(f"{cell['execution_count']} > {p} == {contig}")

55 > 55 == True
56 > 56 == True
58 > 58 == True
59 > 59 == True
60 > 60 == True


In [11]:
#% ignore
compile_notebook("CJN.ipynb", "CJN.py", opts={'compile_marked': True, 'enforce_exec_count': True})

In [12]:
#% ignore
save_compscript({'CJN.ipynb': {'output': 'CJN.py', 'opts': {'compile_marked': False}}}, 'compile.json')
run_compile_script('compile.json')

NameError: name 'output' is not defined